In [12]:
from google.colab import drive
drive.mount('/content/drive')  # This will ask for Google Drive access


Mounted at /content/drive


In [13]:
import os

# Define dataset path inside Google Drive
base_path = "/content/drive/MyDrive/COVID/"

# Use local folder names, not URLs
frame_path = os.path.join(base_path, "frames")  # CT scan images folder
mask_path = os.path.join(base_path, "masks")    # Segmentation masks folder

# Check if dataset folders exist
if not os.path.exists(frame_path):
    print("❌ Error: CT scan images folder ('frames') not found!")
if not os.path.exists(mask_path):
    print("❌ Error: Segmentation masks folder ('masks') not found!")

# List and count files if folders exist
if os.path.exists(frame_path) and os.path.exists(mask_path):
    print(f"✅ Total CT scan images: {len(os.listdir(frame_path))}")
    print(f"✅ Total segmentation masks: {len(os.listdir(mask_path))}")


✅ Total CT scan images: 2833
✅ Total segmentation masks: 2729


In [14]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Dataset Paths
frame_path = "/content/drive/MyDrive/COVID/frames"
mask_path = "/content/drive/MyDrive/COVID/masks"

IMG_SIZE = (256, 256)

# Load images and masks
def load_dataset(image_folder, mask_folder):
    images, masks = [], []

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        mask_path = os.path.join(mask_folder, filename)

        if os.path.exists(mask_path):  # Ensure mask exists for each image
            img = load_img(img_path, target_size=IMG_SIZE, color_mode="grayscale")
            img = img_to_array(img) / 255.0  # Normalize

            mask = load_img(mask_path, target_size=IMG_SIZE, color_mode="grayscale")
            mask = img_to_array(mask) / 255.0  # Normalize

            images.append(img)
            masks.append(mask)

    return np.array(images), np.array(masks)

# Load Data
train_images, train_masks = load_dataset(frame_path, mask_path)

print(f"✅ Loaded {train_images.shape[0]} images and {train_masks.shape[0]} masks.")


✅ Loaded 2729 images and 2729 masks.


In [11]:
!ls /content/drive/MyDrive/COVID


ls: cannot access '/content/drive/MyDrive/COVID': No such file or directory


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

def unet_resnet(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = Concatenate()([conv3, up5])
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    merge6 = Concatenate()([conv2, up6])
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = Concatenate()([conv1, up7])
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge7)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv7)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

images = np.random.rand(100, 256, 256, 1)
masks = np.random.randint(0, 2, (100, 256, 256, 1))

model = unet_resnet()
model.fit(images, masks, epochs=10, batch_size=8, validation_split=0.2)
model.save("model/model_weights.h5")
print("Model trained and saved successfully!")

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 411s 41s/step - accuracy: 0.5002 - loss: 0.6935 - val_accuracy: 0.4995 - val_loss: 0.6932
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 442s 41s/step - accuracy: 0.4997 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 450s 42s/step - accuracy: 0.5002 - loss: 0.6931 - val_accuracy: 0.4996 - val_loss: 0.6932
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 440s 42s/step - accuracy: 0.5000 - loss: 0.6931 - val_accuracy: 0.4996 - val_loss: 0.6932
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 434s 41s/step - accuracy: 0.4999 - loss: 0.6931 - val_accuracy: 0.4998 - val_loss: 0.6931
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 452s 42s/step - accuracy: 0.5007 - loss: 0.6931 - val_accuracy: 0.4996 - val_loss: 0.6931
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 438s 41s/step - accuracy: 0.5011 - loss: 0.6931 - val_accuracy: 0.5007 - val_loss: 0.6931
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 436s 41s/step - accuracy: 0.5006 - loss: 0.6931 - val_accuracy: 0.

Model trained and saved successfully!


In [17]:
model = unet_resnet()
model.fit(train_images, train_masks, epochs=10, batch_size=8, validation_split=0.2)

# Save the model
model.save("/content/drive/MyDrive/COVID/model_weights.h5")
print("✅ Model trained and saved successfully!")


Epoch 1/10
  1/273 ━━━━━━━━━━━━━━━━━━━━ 3:57:35 52s/step - accuracy: 0.1893 - loss: 0.6991

KeyboardInterrupt: 

In [18]:
from fastapi import FastAPI, UploadFile, File
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
from io import BytesIO

app = FastAPI()
model = tf.keras.models.load_model("/content/drive/MyDrive/COVID/model_weights.h5")

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = load_img(BytesIO(await file.read()), target_size=(256, 256), color_mode="grayscale")
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)[0]
    segmented = (prediction > 0.5).astype(np.uint8) * 255

    _, buffer = cv2.imencode(".png", segmented)
    return {"segmented_image": buffer.tobytes()}


ModuleNotFoundError: No module named 'fastapi'

In [1]:
import streamlit as st
import requests
st.title("COVID-19 CT Scan Segmentation")

uploaded_file = st.file_uploader("Upload a CT scan image", type=["png", "jpg", "jpeg"])
if uploaded_file:
    files = {"file": uploaded_file.getvalue()}
    response = requests.post("http://localhost:8000/predict/", files=files)
    if response.status_code == 200:
        st.image(response.json()["segmented_image"], caption="Segmented Output")

ModuleNotFoundError: No module named 'streamlit'